In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Descomprimindo para torna-los csv

In [ ]:
import zipfile

def extract_func(zipped_data_path: str, unzipped_directory: str) -> None:
    with zipfile.ZipFile(zipped_data_path, 'r') as zip_ref:
        zip_ref.extractall(unzipped_directory)

files = ['features.csv.zip', 'sampleSubmission.csv.zip', 'test.csv.zip', 'train.csv.zip']

data_path = '../input/walmart-recruiting-store-sales-forecasting/'
directory_to_extract_to = 'unzip/'

[
    extract_func(
        zipped_data_path=data_path+file,
        unzipped_directory=directory_to_extract_to
    )
    for file in files
]

Depois de descomprimir comecei uma análise exploratoria para entender meu dataset e como usa-lo

In [ ]:
raw_data = pd.read_csv('unzip/train.csv')
raw_data.tail(10)

Verificando se o tamanho da loja é relacionado com o tipo dela, aparentemente podemos dizer que sim

In [ ]:
stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
stores = stores.sort_values(by='Size',ascending=True)

avg_size_store_per_type = stores[['Type','Size']].groupby(['Type']).median()
sns.barplot(x=avg_size_store_per_type.index,y=avg_size_store_per_type.Size)

Verificando abaixo, se a quantidade dos dados sobre as lojas está bem distribuida ao longo do tempo, e se não existe um "gap" em alguma parte da série temporal

In [ ]:
import seaborn as sns

raw_data.sort_values(by=['Date'],ascending=True)
registers_per_day = raw_data[['Store','Dept','Date']].groupby(['Date']).count()
registers_per_day
raw_data[raw_data.Date == '2010-02-05']

sns.barplot(x=registers_per_day.index,y=registers_per_day.Store)

Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

Feriados tem sim um maior volume de vendas que semana sem feriados, porem essa diferença é menor que o esperado

In [ ]:
holiday_evaluation = raw_data[['Weekly_Sales','IsHoliday']].groupby(['IsHoliday']).median()
holiday_evaluation
sns.barplot(x=holiday_evaluation.index,y=holiday_evaluation.Weekly_Sales)

Selecione algumas lojas para conseguir ver o comportamento de cada uma, para entender tendencias e discrepancias entre elas

In [ ]:
selected_stores = raw_data[raw_data.Store > 20]
selected_stores = selected_stores[selected_stores.Store < 40]

selected_stores['Year'] = pd.to_datetime(selected_stores.Date).dt.isocalendar().year
selected_stores['Week'] = pd.to_datetime(selected_stores.Date).dt.isocalendar().week

g = sns.FacetGrid(selected_stores, col="Store", col_wrap=5)
g.map(sns.lineplot, "Week", "Weekly_Sales")

A maioria das lojas tem o mesmo pico de vendas no fim do ano, mas algumas lojas não possuem essa variabilidade, provavelmente atrelada ao tipo de loja que estamos falando, mas que precisa ser validada pelos dados

In [ ]:
g = sns.FacetGrid(selected_stores, col="Year", col_wrap=5)
g.map(sns.lineplot, "Week", "Weekly_Sales")

Mesmo tendo 4 feriados no conjunto de dados, podemos ver que existem dois picos muito maiores no fim do ano, ou seja, alguns feriados possuem mais importancia que outras quando falamos de venda.
Uma hipotese, é que os descontos tambem sejam maiores por volta dessas datas.
Ou seja, esses feriados de fim de ano podem ser usados ao nosso favor quando modelando, fazendo assim que eles se tornem uma feature do modelo

In [ ]:
end_of_the_year = selected_stores[selected_stores.Week > 45]

g = sns.FacetGrid(end_of_the_year, col="Year", col_wrap=5)
g.map(sns.lineplot, "Week", "Weekly_Sales")

Vemos dois pontos que se destacam, são eles Natal e Dia de Ação de Graças, que tem uma importancia na venda ainda maior que os outros feriados observados

Abaixo estou tentando olhar para os indicatores que o desafio oferece, vendo se algum deles trás alguma informação sobre minha váriavel alvo

In [ ]:
additional_features = pd.read_csv('unzip/features.csv')
additional_features['Year'] = pd.to_datetime(additional_features.Date).dt.isocalendar().year
additional_features['Week'] = pd.to_datetime(additional_features.Date).dt.isocalendar().week

raw_data['Year'] = pd.to_datetime(raw_data.Date).dt.isocalendar().year
raw_data['Week'] = pd.to_datetime(raw_data.Date).dt.isocalendar().week

Fuel_vs_Sales = pd.merge(
    left=raw_data,
    right=additional_features,
    how='left',
    left_on=['Store', 'Year', 'Week'],
    right_on=['Store', 'Year', 'Week']
)

sns.lineplot(x=Fuel_vs_Sales.Fuel_Price,y=Fuel_vs_Sales.Weekly_Sales)

Preço do combustivel não se mostrou uma variavel 

In [ ]:
sns.lineplot(x=Fuel_vs_Sales.Unemployment,y=Fuel_vs_Sales.Weekly_Sales)

Desemprego não mostra uma correlação tão "limpa" com as vendas, porem pode existir algo ali no meio que possa potencializar isso, porem precisaria de mais tempo investigando essa variável para achar a maneira correta de utilizar no modelo

In [ ]:
experimentation_set = pd.read_csv('unzip/train.csv')
stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')

full_set = pd.merge(
    left=experimentation_set,
    right=stores,
    how='left',
    left_on='Store',
    right_on='Store'
)

full_set['Week'] = pd.to_datetime(full_set.Date).dt.isocalendar().week
full_set['Year'] = pd.to_datetime(full_set.Date).dt.isocalendar().year
full_set['Date'] = pd.to_datetime(full_set.Date)

In [ ]:
evaluation_set = pd.read_csv('unzip/test.csv')

additional_features.dropna()
additional_features.sort_values(by='Date',ascending=False)

g = sns.FacetGrid(additional_features, col="Year")
g.map(sns.lineplot, "Week", "MarkDown1")

g = sns.FacetGrid(full_set, col="Year")
g.map(sns.lineplot, "Week", "Weekly_Sales")

Os markdowns são the categorias diferentes, ou seja, cada um provavelmente tem sua peculiaridade e relação com as vendas finais da semana
Assumindo que as venda são uma série temporal e o comportamento passado se assemelha ao do futuro, decidi tentar modelar esse markdown como uma porcentagem, ou seja, olhando para o passado, em relação ao periodo de uma ano, qual a porcentagem de markdown normalmente acontece naquele período do ano em relação ao maximo de markdown possivel.

In [ ]:
markdowns = ["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5","Week"]

enginnered_markdown = additional_features[markdowns].groupby(by="Week").median()
enginnered_markdown = enginnered_markdown/enginnered_markdown.max()
enginnered_markdown = enginnered_markdown.fillna(0)

enginnered_markdown

Após modelar o MarkDown, decidi adicionar as features relacionadas as descobertas de maiores vendas no natal e na época de acão de graças

In [ ]:
full_set = pd.merge(
    left=full_set,
    right=enginnered_markdown,
    how="left",
    left_on="Week",
    right_on="Week"
)

full_set['IsChristmas'] = full_set['Week'].apply(lambda x: 1 if x == 51 else 0)

full_set['IsThanksgiving'] = full_set['Week'].apply(lambda x: 1 if x == 47 else 0)

In [ ]:
full_set

Essa é uma simples alteração, fazendo com que as categorias virem número para que possam alimentar o modelo. Poderia ter feito um one hot encoding aqui, mas como elas tem uma relação de ordem entre elas, ou seja, uma maior que a outra, preferi essa solução para preservar esta propriedade

In [ ]:
def categories_to_ordinal(categorie: str):
    if categorie == 'A':
        return 1
    if categorie == 'B':
        return 2
    return 3

full_set['ModType'] = full_set['Type'].apply(lambda categorie: categories_to_ordinal(categorie)) 

Como os dados aqui são limitados, selecione para o meu conjunto de treino tudo que não é 2011, e para teste o ano todo de 2011. Fiz isso para que conseguisse ter confiança em todas as partes do ano que estou realizando a predição. Preservando uma série histórica toda para a validação

In [ ]:
train_set = full_set[full_set.Year != 2011]
test_set = full_set[full_set.Year == 2011]

features = ['Store','Dept','Week','IsHoliday','ModType', 'IsChristmas','IsThanksgiving', 'MarkDown1' ,'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
target = 'Weekly_Sales'

X_train = train_set[features]
y_train = train_set[target]

X_test = test_set[features]
y_real = test_set[target]

Esse abaixo é o codigo para conseguir observar o erro ponderado dos modelos treinados, aproximando a performance segundo o que foi dito no desafio

In [ ]:
def WMAE(dataset, real, predicted):
    weights = dataset.IsHoliday.apply(lambda weight: 5 if weight else 1)
    return np.round(np.sum(weights*abs(real-predicted))/(np.sum(weights)), 2)

O primeiro modelo testado foi o random forest, com uma variação no parametro de profundidade.
Até 25 ele mostra uma melhora, depois se torna insignificante ou até pior.
Resolvi começar por random forest pois estava com medo dos resultados dos dias de feriado, acreditava que esse modelo pudesse contornar isso, visto que ele lida bem com esses *outliers*

In [ ]:
from sklearn.ensemble import RandomForestRegressor
max_depth_options = list(range(20,25))

for depth in max_depth_options:
    regr = RandomForestRegressor(max_depth=depth, random_state=0)
    regr.fit(X_train, y_train)
    y_predicted = regr.predict(X_test)
    print(f"Depth {depth}: " + str(WMAE(X_test, y_real, y_predicted)))

In [ ]:
sns.lineplot(x=test_set.Date,y=y_real)
sns.lineplot(x=test_set.Date,y=y_predicted)

Depois de testar esse primeiro modelo, resolvi ir para o xgboost.
Ambos fazem uso de ensamble learning (combinação de dois ou mais modelos para obter um melhor) e decision trees mas agora usando gradient boosting (combina os modelos sequencialmente) e não bagging (primeiro se criam varios subsets das amostras do dataset de treino, se treinam modelos nessas amostras, por fim uma média é criada para gerar uma estimativa mais assertiva) como no modelo de random forest.

In [ ]:
import xgboost as xgb

regressor = xgb.XGBRegressor(
    n_estimators=120,
    reg_lambda=1,
    gamma=0,
    max_depth=25
)

X_train['Week'] = X_train['Week'].astype('int')

regressor.fit(X_train, y_train)

In [ ]:
X_test['Week'] = X_test['Week'].astype('int')

y_pred = regressor.predict(X_test)

In [ ]:
sns.lineplot(x=test_set.Date,y=y_real)
sns.lineplot(x=test_set.Date,y=y_pred)

In [ ]:
WMAE(X_test, y_real, y_pred)

A diferença entre eles ficou bem baixa, então para mim a escolha ficaria a cargo de performance de treino/escalabilidade, sendo assim acredito que para um ambinete de produção usar o XGBoost seria melhor devido a implementação da biblioteca